# Runtime Performance Comparison: Node.js vs Deno vs Bun

This notebook provides a comprehensive comparison of Next.js application performance across three JavaScript runtimes:
- **Node.js**: The traditional JavaScript runtime
- **Deno**: A secure runtime for JavaScript and TypeScript  
- **Bun**: A fast all-in-one JavaScript runtime

## Overview

We'll create identical Next.js applications for each runtime, containerize them using Docker, and monitor their CPU and memory usage patterns to determine which runtime performs best under different scenarios.

## Prerequisites

- Docker and Docker Compose installed
- Python 3.11+ with required packages
- Sufficient system resources to run multiple containers

## 1. Setup Project Structure

First, let's verify our project structure and import the necessary libraries for performance monitoring and data analysis.

In [ ]:
import os
import sys
import json
import time
import subprocess
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import warnings
from datetime import datetime, timedelta
import threading
from IPython.display import display, HTML, clear_output
import ipywidgets as widgets

# Import our custom performance monitor
sys.path.append('/app')
from performance_monitor import PerformanceMonitor

# Configure plotting
plt.style.use('default')
sns.set_palette("husl")
warnings.filterwarnings('ignore')

# Set up display options
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)

print("✅ All libraries imported successfully!")
print(f"📁 Working directory: {os.getcwd()}")
print(f"🐍 Python version: {sys.version}")
print(f"🕐 Current time: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")

In [ ]:
# Verify project structure
project_root = "/Users/jabbo/node-vs-deno-vs-bun"
expected_dirs = ["node-nextjs", "deno-nextjs", "bun-nextjs", "monitoring"]

print("🔍 Checking project structure...")
for directory in expected_dirs:
    path = os.path.join(project_root, directory)
    if os.path.exists(path):
        print(f"✅ {directory}/ exists")
        # List key files in each directory
        files = os.listdir(path)
        key_files = [f for f in files if f in ['package.json', 'deno.json', 'Dockerfile', 'next.config.js']]
        if key_files:
            print(f"   📄 Key files: {', '.join(key_files)}")
    else:
        print(f"❌ {directory}/ missing")

# Check if docker-compose.yml exists
compose_file = os.path.join(project_root, "docker-compose.yml")
if os.path.exists(compose_file):
    print("✅ docker-compose.yml exists")
else:
    print("❌ docker-compose.yml missing")

## 2. Create NextJS Applications

Our project structure includes three identical Next.js applications, each configured for a different runtime:

1. **node-nextjs/**: Traditional Node.js setup with npm
2. **deno-nextjs/**: Deno setup with deno.json configuration  
3. **bun-nextjs/**: Bun setup with native Bun package manager

Each application runs on a different port:
- Node.js: `http://localhost:3001`
- Deno: `http://localhost:3002` 
- Bun: `http://localhost:3003`

In [ ]:
# Display configuration comparison
configurations = {
    "Runtime": ["Node.js", "Deno", "Bun"],
    "Package Manager": ["npm", "deno", "bun"],
    "Port": [3001, 3002, 3003],
    "Config File": ["package.json", "deno.json", "package.json"],
    "Container Name": ["node-nextjs-app", "deno-nextjs-app", "bun-nextjs-app"]
}

config_df = pd.DataFrame(configurations)
print("📋 Runtime Configuration Summary:")
display(config_df)

# Check package.json files to verify Next.js versions
print("\n🔍 Checking Next.js versions across runtimes:")
for runtime in ["node-nextjs", "deno-nextjs", "bun-nextjs"]:
    package_path = f"{project_root}/{runtime}/package.json"
    if os.path.exists(package_path):
        with open(package_path, 'r') as f:
            package_data = json.load(f)
            nextjs_version = package_data.get('dependencies', {}).get('next', 'Not found')
            print(f"  {runtime}: Next.js {nextjs_version}")
    else:
        print(f"  {runtime}: package.json not found")

## 3. Generate Dockerfiles for Each Runtime

Each runtime has its own optimized Dockerfile:

- **Node.js**: Uses `node:18-alpine` base image with npm
- **Deno**: Uses `denoland/deno:1.37.0` with native Deno commands
- **Bun**: Uses `oven/bun:1.0.7` with native Bun commands

All containers are configured for production builds with the `standalone` output mode.

In [ ]:
# Display Dockerfile comparison
print("🐳 Dockerfile Configuration Comparison:\n")

dockerfiles = {
    "node-nextjs": f"{project_root}/node-nextjs/Dockerfile",
    "deno-nextjs": f"{project_root}/deno-nextjs/Dockerfile", 
    "bun-nextjs": f"{project_root}/bun-nextjs/Dockerfile"
}

dockerfile_info = []
for runtime, dockerfile_path in dockerfiles.items():
    if os.path.exists(dockerfile_path):
        with open(dockerfile_path, 'r') as f:
            content = f.read()
            
        # Extract key information
        lines = content.split('\n')
        base_image = next((line.split()[1] for line in lines if line.startswith('FROM')), 'Not found')
        expose_port = next((line.split()[1] for line in lines if line.startswith('EXPOSE')), 'Not found')
        
        dockerfile_info.append({
            'Runtime': runtime,
            'Base Image': base_image,
            'Exposed Port': expose_port,
            'File Size (bytes)': len(content)
        })
        
        print(f"📄 {runtime}/Dockerfile:")
        print(f"   Base Image: {base_image}")
        print(f"   Exposed Port: {expose_port}")
        print(f"   File Size: {len(content)} bytes")
        print()
    else:
        print(f"❌ {dockerfile_path} not found")

# Create DataFrame for comparison
if dockerfile_info:
    dockerfile_df = pd.DataFrame(dockerfile_info)
    display(dockerfile_df)

## 4. Create Docker Compose Configuration

The Docker Compose file orchestrates all three Next.js applications plus our monitoring container:

- **Services**: node-nextjs, deno-nextjs, bun-nextjs, monitoring
- **Network**: All containers share the `nextjs-comparison` network
- **Health Checks**: Each app container has HTTP health checks
- **Volumes**: Monitoring container has access to Docker socket and shared data volumes

In [ ]:
# Check Docker and Docker Compose availability
def check_docker():
    try:
        result = subprocess.run(['docker', '--version'], capture_output=True, text=True)
        if result.returncode == 0:
            print(f"✅ Docker: {result.stdout.strip()}")
            return True
        else:
            print("❌ Docker not found")
            return False
    except FileNotFoundError:
        print("❌ Docker command not found")
        return False

def check_docker_compose():
    try:
        result = subprocess.run(['docker-compose', '--version'], capture_output=True, text=True)
        if result.returncode == 0:
            print(f"✅ Docker Compose: {result.stdout.strip()}")
            return True
        else:
            # Try docker compose (newer syntax)
            result = subprocess.run(['docker', 'compose', 'version'], capture_output=True, text=True)
            if result.returncode == 0:
                print(f"✅ Docker Compose: {result.stdout.strip()}")
                return True
            else:
                print("❌ Docker Compose not found")
                return False
    except FileNotFoundError:
        print("❌ Docker Compose command not found")
        return False

print("🔍 Checking Docker environment:")
docker_available = check_docker()
compose_available = check_docker_compose()

if docker_available and compose_available:
    print("\n✅ Docker environment is ready!")
else:
    print("\n⚠️  Docker environment setup required before proceeding")

# Display docker-compose.yml structure
compose_path = f"{project_root}/docker-compose.yml"
if os.path.exists(compose_path):
    print(f"\n📄 Docker Compose file structure:")
    with open(compose_path, 'r') as f:
        lines = f.readlines()
    
    # Show service definitions
    services = []
    for line in lines:
        if line.strip().endswith(':') and not line.startswith(' ') and 'services' not in line:
            service_name = line.strip().rstrip(':')
            if service_name not in ['version', 'networks']:
                services.append(service_name)
    
    print(f"   Services defined: {', '.join(services)}")
    print(f"   Total lines: {len(lines)}")
else:
    print(f"\n❌ {compose_path} not found")

## 5. Build Performance Monitoring Script

Our performance monitoring system includes:

- **PerformanceMonitor class**: Monitors Docker containers using the Docker API
- **Metrics collection**: CPU percentage, memory usage (MB and %), container health
- **Data persistence**: Automatic saving to JSON and CSV formats
- **Real-time monitoring**: Configurable interval-based collection
- **System-wide metrics**: Overall system resource usage

In [ ]:
# Initialize performance monitor
monitor = PerformanceMonitor(interval=5)  # Monitor every 5 seconds

print("🔧 Performance Monitor Configuration:")
print(f"   Monitoring interval: {monitor.interval} seconds")
print(f"   Target containers: {list(monitor.containers.keys())}")
print(f"   Data storage path: /app/data/")

# Test monitor connectivity
print("\n🔍 Testing Docker connectivity:")
try:
    import docker
    client = docker.from_env()
    containers = client.containers.list()
    print(f"   ✅ Docker client connected")
    print(f"   📦 Currently running containers: {len(containers)}")
    
    for container in containers:
        print(f"     - {container.name}: {container.status}")
        
except Exception as e:
    print(f"   ❌ Docker connection failed: {e}")

# Show monitor methods
print("\n📋 Available monitoring methods:")
methods = [method for method in dir(monitor) if not method.startswith('_') and callable(getattr(monitor, method))]
for method in methods[:10]:  # Show first 10 methods
    print(f"   - {method}()")
    
print(f"   ... and {len(methods)-10} more methods" if len(methods) > 10 else "")

## 6. Run Docker Containers and Collect Metrics

This section will start the Docker containers and begin performance monitoring. We'll:

1. **Build and start all containers** using Docker Compose
2. **Wait for containers to be healthy** before starting monitoring
3. **Collect performance metrics** for a specified duration
4. **Save the data** for analysis

**⚠️ Important**: This step requires the containers to be running. If you're running this notebook inside the monitoring container, the other containers should already be started via Docker Compose.

In [ ]:
# Check container status and health
def check_container_status():
    """Check the status of all target containers"""
    import docker
    client = docker.from_env()
    
    container_status = {}
    for container_name in monitor.containers.keys():
        try:
            container = client.containers.get(container_name)
            container_status[container_name] = {
                'status': container.status,
                'health': getattr(container.attrs['State'], 'Health', {}).get('Status', 'N/A'),
                'ports': container.ports
            }
        except docker.errors.NotFound:
            container_status[container_name] = {
                'status': 'not found',
                'health': 'N/A',
                'ports': {}
            }
        except Exception as e:
            container_status[container_name] = {
                'status': f'error: {e}',
                'health': 'N/A',
                'ports': {}
            }
    
    return container_status

# Check current container status
print("🔍 Checking container status:")
status = check_container_status()

status_df_data = []
for name, info in status.items():
    status_df_data.append({
        'Container': name,
        'Status': info['status'],
        'Health': info['health'],
        'Ports': str(info['ports']) if info['ports'] else 'None'
    })

status_df = pd.DataFrame(status_df_data)
display(status_df)

# Check if containers are ready for monitoring
ready_containers = [name for name, info in status.items() if info['status'] == 'running']
print(f"\n📊 Containers ready for monitoring: {len(ready_containers)}/{len(monitor.containers)}")

if len(ready_containers) < len(monitor.containers):
    print("⚠️  Some containers are not running. You may need to start them with:")
    print("   docker-compose up -d")
else:
    print("✅ All containers are running and ready for monitoring!")

In [ ]:
# Interactive monitoring controls
monitoring_duration = widgets.IntSlider(
    value=60,
    min=30,
    max=300,
    step=10,
    description='Duration (s):',
    disabled=False
)

monitoring_interval = widgets.IntSlider(
    value=5,
    min=1,
    max=30,
    step=1,
    description='Interval (s):',
    disabled=False
)

start_button = widgets.Button(
    description='Start Monitoring',
    disabled=False,
    button_style='success',
    tooltip='Start performance monitoring',
    icon='play'
)

stop_button = widgets.Button(
    description='Stop Monitoring',
    disabled=True,
    button_style='danger',
    tooltip='Stop performance monitoring',
    icon='stop'
)

output_widget = widgets.Output()

# Monitoring state
monitoring_active = False
monitoring_thread = None

def start_monitoring(button):
    global monitoring_active, monitoring_thread
    
    if monitoring_active:
        return
    
    # Update monitor settings
    monitor.interval = monitoring_interval.value
    
    # Disable start button, enable stop button
    start_button.disabled = True
    stop_button.disabled = False
    
    with output_widget:
        clear_output(wait=True)
        print(f"🚀 Starting monitoring for {monitoring_duration.value} seconds...")
        print(f"📊 Monitoring interval: {monitoring_interval.value} seconds")
        print("🔄 Monitoring in progress...")
    
    monitoring_active = True
    
    # Start monitoring in a separate thread
    def monitor_task():
        monitor.start_monitoring(duration=monitoring_duration.value)
        
        # Update UI when done
        start_button.disabled = False
        stop_button.disabled = True
        
        with output_widget:
            print("✅ Monitoring completed!")
            print(f"📈 Collected {len(monitor.data)} data points")
    
    monitoring_thread = threading.Thread(target=monitor_task)
    monitoring_thread.daemon = True
    monitoring_thread.start()

def stop_monitoring(button):
    global monitoring_active
    
    if not monitoring_active:
        return
    
    monitor.running = False
    monitoring_active = False
    
    # Update buttons
    start_button.disabled = False
    stop_button.disabled = True
    
    with output_widget:
        print("🛑 Monitoring stopped by user")
        print(f"📈 Collected {len(monitor.data)} data points")

start_button.on_click(start_monitoring)
stop_button.on_click(stop_monitoring)

# Display controls
print("🎛️  Monitoring Controls:")
display(widgets.VBox([
    widgets.HBox([monitoring_duration, monitoring_interval]),
    widgets.HBox([start_button, stop_button]),
    output_widget
]))

## 7. Analyze Performance Data

After monitoring, we'll analyze the collected performance data to understand:

- **CPU Usage Patterns**: How each runtime utilizes CPU resources
- **Memory Consumption**: Memory usage patterns and efficiency  
- **Resource Stability**: Consistency of resource usage over time
- **Health Status**: Application availability and responsiveness
- **Comparative Performance**: Direct comparisons between runtimes

In [ ]:
# Load and analyze performance data
def load_latest_data():
    """Load the most recent performance data"""
    data_dir = "/app/data"
    
    if not os.path.exists(data_dir):
        print("❌ Data directory not found")
        return None
    
    # Find the most recent CSV file
    csv_files = [f for f in os.listdir(data_dir) if f.endswith('.csv')]
    
    if not csv_files:
        # Try to get data from monitor if available
        if hasattr(monitor, 'data') and monitor.data:
            print("📊 Using data from current monitoring session")
            return monitor.get_dataframe()
        else:
            print("❌ No performance data found")
            return None
    
    # Get the most recent file
    latest_file = max(csv_files, key=lambda f: os.path.getctime(os.path.join(data_dir, f)))
    file_path = os.path.join(data_dir, latest_file)
    
    print(f"📈 Loading data from: {latest_file}")
    df = pd.read_csv(file_path)
    df['timestamp'] = pd.to_datetime(df['timestamp'])
    
    return df

# Load performance data
performance_df = load_latest_data()

if performance_df is not None:
    print(f"✅ Loaded {len(performance_df)} data points")
    print(f"📅 Time range: {performance_df['timestamp'].min()} to {performance_df['timestamp'].max()}")
    print(f"⏱️  Duration: {(performance_df['timestamp'].max() - performance_df['timestamp'].min()).total_seconds():.0f} seconds")
    
    # Display basic statistics
    print("\n📋 Data Overview:")
    display(performance_df.head())
    
    print("\n📊 Container Distribution:")
    container_counts = performance_df['container_name'].value_counts()
    display(container_counts)
    
else:
    print("⚠️  No performance data available. Please run monitoring first.")

In [ ]:
# Calculate performance statistics
if performance_df is not None:
    print("📊 Performance Statistics Summary:\n")
    
    # Group by container for analysis
    stats_by_container = performance_df.groupby('container_name').agg({
        'container_cpu_percent': ['mean', 'std', 'min', 'max'],
        'container_memory_usage_mb': ['mean', 'std', 'min', 'max'],
        'container_memory_percent': ['mean', 'std', 'min', 'max'],
        'container_healthy': ['mean']  # Percentage of time healthy
    }).round(2)
    
    # Flatten column names
    stats_by_container.columns = ['_'.join(col).strip() for col in stats_by_container.columns]
    stats_by_container = stats_by_container.rename(columns={
        'container_healthy_mean': 'health_percentage'
    })
    
    display(stats_by_container)
    
    # Create summary comparison
    print("\n🏆 Performance Comparison (Lower is Better for CPU/Memory):")
    
    summary_stats = []
    for container in performance_df['container_name'].unique():
        container_data = performance_df[performance_df['container_name'] == container]
        
        runtime_name = container.replace('-nextjs-app', '').replace('-', ' ').title()
        
        summary_stats.append({
            'Runtime': runtime_name,
            'Avg CPU (%)': container_data['container_cpu_percent'].mean(),
            'Avg Memory (MB)': container_data['container_memory_usage_mb'].mean(),
            'Avg Memory (%)': container_data['container_memory_percent'].mean(),
            'CPU Stability (σ)': container_data['container_cpu_percent'].std(),
            'Memory Stability (σ)': container_data['container_memory_usage_mb'].std(),
            'Health %': (container_data['container_healthy'].mean() * 100),
            'Data Points': len(container_data)
        })
    
    summary_df = pd.DataFrame(summary_stats).round(2)
    
    # Sort by average CPU usage
    summary_df = summary_df.sort_values('Avg CPU (%)')
    display(summary_df)
    
    # Determine winners
    print("\n🥇 Performance Winners:")
    print(f"   Lowest CPU Usage: {summary_df.iloc[0]['Runtime']} ({summary_df.iloc[0]['Avg CPU (%)']}%)")
    print(f"   Lowest Memory Usage: {summary_df.loc[summary_df['Avg Memory (MB)'].idxmin(), 'Runtime']} ({summary_df['Avg Memory (MB)'].min():.1f} MB)")
    print(f"   Most Stable CPU: {summary_df.loc[summary_df['CPU Stability (σ)'].idxmin(), 'Runtime']} (σ = {summary_df['CPU Stability (σ)'].min():.2f})")
    print(f"   Best Health Score: {summary_df.loc[summary_df['Health %'].idxmax(), 'Runtime']} ({summary_df['Health %'].max():.1f}%)")
    
else:
    print("❌ Cannot calculate statistics - no performance data available")

## 8. Visualize Results

Create comprehensive visualizations to compare the performance characteristics of each runtime:

- **Time Series Plots**: CPU and memory usage over time
- **Box Plots**: Distribution of resource usage
- **Bar Charts**: Average performance metrics comparison
- **Correlation Analysis**: Relationship between CPU and memory usage
- **Performance Dashboard**: Interactive multi-metric view

In [ ]:
# Create comprehensive visualizations
if performance_df is not None:
    
    # Set up color palette for runtimes
    runtime_colors = {
        'node-nextjs-app': '#68A063',    # Node.js green
        'deno-nextjs-app': '#000000',    # Deno black  
        'bun-nextjs-app': '#FBF0DF'      # Bun cream
    }
    
    # 1. Time Series Plot - CPU Usage
    fig = make_subplots(
        rows=2, cols=2,
        subplot_titles=('CPU Usage Over Time', 'Memory Usage Over Time', 
                       'CPU Usage Distribution', 'Memory Usage Distribution'),
        specs=[[{"secondary_y": False}, {"secondary_y": False}],
               [{"secondary_y": False}, {"secondary_y": False}]]
    )
    
    # CPU time series
    for container in performance_df['container_name'].unique():
        container_data = performance_df[performance_df['container_name'] == container]
        runtime_name = container.replace('-nextjs-app', '').replace('-', ' ').title()
        
        fig.add_trace(
            go.Scatter(
                x=container_data['timestamp'],
                y=container_data['container_cpu_percent'],
                mode='lines+markers',
                name=f'{runtime_name} CPU',
                line=dict(color=runtime_colors.get(container, '#1f77b4')),
                showlegend=True
            ),
            row=1, col=1
        )
        
        # Memory time series
        fig.add_trace(
            go.Scatter(
                x=container_data['timestamp'],
                y=container_data['container_memory_usage_mb'],
                mode='lines+markers',
                name=f'{runtime_name} Memory',
                line=dict(color=runtime_colors.get(container, '#1f77b4'), dash='dot'),
                showlegend=True
            ),
            row=1, col=2
        )
        
        # CPU distribution
        fig.add_trace(
            go.Box(
                y=container_data['container_cpu_percent'],
                name=f'{runtime_name}',
                marker_color=runtime_colors.get(container, '#1f77b4'),
                showlegend=False
            ),
            row=2, col=1
        )
        
        # Memory distribution  
        fig.add_trace(
            go.Box(
                y=container_data['container_memory_usage_mb'],
                name=f'{runtime_name}',
                marker_color=runtime_colors.get(container, '#1f77b4'),
                showlegend=False
            ),
            row=2, col=2
        )
    
    # Update layout
    fig.update_layout(
        height=800,
        title_text="Next.js Runtime Performance Comparison",
        title_x=0.5,
        showlegend=True
    )
    
    # Update axes labels
    fig.update_xaxes(title_text="Time", row=1, col=1)
    fig.update_xaxes(title_text="Time", row=1, col=2)
    fig.update_xaxes(title_text="Runtime", row=2, col=1)
    fig.update_xaxes(title_text="Runtime", row=2, col=2)
    
    fig.update_yaxes(title_text="CPU Usage (%)", row=1, col=1)
    fig.update_yaxes(title_text="Memory Usage (MB)", row=1, col=2)
    fig.update_yaxes(title_text="CPU Usage (%)", row=2, col=1)
    fig.update_yaxes(title_text="Memory Usage (MB)", row=2, col=2)
    
    fig.show()
    
else:
    print("❌ Cannot create visualizations - no performance data available")

In [ ]:
# Create performance comparison charts
if performance_df is not None:
    
    # Prepare data for comparison charts
    comparison_data = []
    for container in performance_df['container_name'].unique():
        container_data = performance_df[performance_df['container_name'] == container]
        runtime_name = container.replace('-nextjs-app', '').replace('-', ' ').title()
        
        comparison_data.append({
            'Runtime': runtime_name,
            'Avg_CPU': container_data['container_cpu_percent'].mean(),
            'Avg_Memory_MB': container_data['container_memory_usage_mb'].mean(),
            'Avg_Memory_Pct': container_data['container_memory_percent'].mean(),
            'CPU_Std': container_data['container_cpu_percent'].std(),
            'Memory_Std': container_data['container_memory_usage_mb'].std()
        })
    
    comparison_df = pd.DataFrame(comparison_data)
    
    # Create matplotlib subplots
    fig, axes = plt.subplots(2, 2, figsize=(15, 10))
    fig.suptitle('Performance Comparison: Node.js vs Deno vs Bun', fontsize=16, fontweight='bold')
    
    # Color scheme
    colors = ['#68A063', '#000000', '#FBF0DF']  # Node, Deno, Bun
    
    # 1. Average CPU Usage
    bars1 = axes[0, 0].bar(comparison_df['Runtime'], comparison_df['Avg_CPU'], color=colors)
    axes[0, 0].set_title('Average CPU Usage (%)')
    axes[0, 0].set_ylabel('CPU Usage (%)')
    axes[0, 0].tick_params(axis='x', rotation=45)
    
    # Add value labels on bars
    for bar, value in zip(bars1, comparison_df['Avg_CPU']):
        axes[0, 0].text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.1,
                       f'{value:.1f}%', ha='center', va='bottom')
    
    # 2. Average Memory Usage
    bars2 = axes[0, 1].bar(comparison_df['Runtime'], comparison_df['Avg_Memory_MB'], color=colors)
    axes[0, 1].set_title('Average Memory Usage (MB)')
    axes[0, 1].set_ylabel('Memory Usage (MB)')
    axes[0, 1].tick_params(axis='x', rotation=45)
    
    # Add value labels on bars
    for bar, value in zip(bars2, comparison_df['Avg_Memory_MB']):
        axes[0, 1].text(bar.get_x() + bar.get_width()/2, bar.get_height() + 1,
                       f'{value:.0f}MB', ha='center', va='bottom')
    
    # 3. CPU Stability (Lower standard deviation = more stable)
    bars3 = axes[1, 0].bar(comparison_df['Runtime'], comparison_df['CPU_Std'], color=colors)
    axes[1, 0].set_title('CPU Usage Stability (Lower = Better)')
    axes[1, 0].set_ylabel('Standard Deviation')
    axes[1, 0].tick_params(axis='x', rotation=45)
    
    # Add value labels on bars
    for bar, value in zip(bars3, comparison_df['CPU_Std']):
        axes[1, 0].text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.01,
                       f'{value:.2f}', ha='center', va='bottom')
    
    # 4. Memory Stability
    bars4 = axes[1, 1].bar(comparison_df['Runtime'], comparison_df['Memory_Std'], color=colors)
    axes[1, 1].set_title('Memory Usage Stability (Lower = Better)')
    axes[1, 1].set_ylabel('Standard Deviation (MB)')
    axes[1, 1].tick_params(axis='x', rotation=45)
    
    # Add value labels on bars
    for bar, value in zip(bars4, comparison_df['Memory_Std']):
        axes[1, 1].text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.5,
                       f'{value:.1f}MB', ha='center', va='bottom')
    
    plt.tight_layout()
    plt.show()
    
    # Performance ranking
    print("🏆 Performance Rankings:\n")
    
    # Rank by CPU usage (lower is better)
    cpu_ranking = comparison_df.sort_values('Avg_CPU')[['Runtime', 'Avg_CPU']]
    print("CPU Usage (Lower is Better):")
    for i, (_, row) in enumerate(cpu_ranking.iterrows(), 1):
        medal = "🥇" if i == 1 else "🥈" if i == 2 else "🥉"
        print(f"  {medal} {i}. {row['Runtime']}: {row['Avg_CPU']:.2f}%")
    
    print()
    
    # Rank by memory usage (lower is better)
    memory_ranking = comparison_df.sort_values('Avg_Memory_MB')[['Runtime', 'Avg_Memory_MB']]
    print("Memory Usage (Lower is Better):")
    for i, (_, row) in enumerate(memory_ranking.iterrows(), 1):
        medal = "🥇" if i == 1 else "🥈" if i == 2 else "🥉"
        print(f"  {medal} {i}. {row['Runtime']}: {row['Avg_Memory_MB']:.1f} MB")
    
else:
    print("❌ Cannot create comparison charts - no performance data available")

In [ ]:
# Create correlation and efficiency analysis
if performance_df is not None:
    
    print("🔬 Advanced Performance Analysis:\n")
    
    # CPU vs Memory correlation analysis
    fig, axes = plt.subplots(1, 2, figsize=(15, 6))
    
    # Scatter plot: CPU vs Memory for each runtime
    for container in performance_df['container_name'].unique():
        container_data = performance_df[performance_df['container_name'] == container]
        runtime_name = container.replace('-nextjs-app', '').replace('-', ' ').title()
        color = runtime_colors.get(container, '#1f77b4')
        
        axes[0].scatter(container_data['container_cpu_percent'], 
                       container_data['container_memory_usage_mb'],
                       label=runtime_name, alpha=0.7, color=color, s=50)
        
        # Calculate correlation
        correlation = container_data['container_cpu_percent'].corr(container_data['container_memory_usage_mb'])
        print(f"{runtime_name} CPU-Memory Correlation: {correlation:.3f}")
    
    axes[0].set_xlabel('CPU Usage (%)')
    axes[0].set_ylabel('Memory Usage (MB)')
    axes[0].set_title('CPU vs Memory Usage Correlation')
    axes[0].legend()
    axes[0].grid(True, alpha=0.3)
    
    # Resource efficiency radar chart (using matplotlib)
    # Calculate efficiency metrics (lower values = better efficiency)
    efficiency_metrics = {}
    for container in performance_df['container_name'].unique():
        container_data = performance_df[performance_df['container_name'] == container]
        runtime_name = container.replace('-nextjs-app', '').replace('-', ' ').title()
        
        # Normalize metrics (0-1 scale, lower is better)
        avg_cpu = container_data['container_cpu_percent'].mean()
        avg_memory = container_data['container_memory_usage_mb'].mean()
        cpu_stability = container_data['container_cpu_percent'].std()
        memory_stability = container_data['container_memory_usage_mb'].std()
        
        efficiency_metrics[runtime_name] = {
            'CPU Usage': avg_cpu,
            'Memory Usage': avg_memory / 100,  # Scale to similar range
            'CPU Stability': cpu_stability,
            'Memory Stability': memory_stability / 10  # Scale to similar range
        }
    
    # Create efficiency comparison
    metric_names = list(efficiency_metrics[list(efficiency_metrics.keys())[0]].keys())
    x_pos = np.arange(len(metric_names))
    width = 0.25
    
    for i, (runtime, metrics) in enumerate(efficiency_metrics.items()):
        values = list(metrics.values())
        color = ['#68A063', '#000000', '#FBF0DF'][i]
        axes[1].bar(x_pos + i * width, values, width, label=runtime, color=color, alpha=0.8)
    
    axes[1].set_xlabel('Metrics')
    axes[1].set_ylabel('Score (Lower = Better)')
    axes[1].set_title('Resource Efficiency Comparison')
    axes[1].set_xticks(x_pos + width)
    axes[1].set_xticklabels(metric_names, rotation=45)
    axes[1].legend()
    axes[1].grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()
    
    # Calculate overall efficiency score
    print("\n📊 Overall Efficiency Scores (Lower = Better):")
    overall_scores = {}
    for runtime, metrics in efficiency_metrics.items():
        # Weighted average (equal weights for simplicity)
        score = sum(metrics.values()) / len(metrics)
        overall_scores[runtime] = score
        
    # Sort by efficiency (lower is better)
    sorted_scores = sorted(overall_scores.items(), key=lambda x: x[1])
    
    for i, (runtime, score) in enumerate(sorted_scores, 1):
        medal = "🥇" if i == 1 else "🥈" if i == 2 else "🥉"
        print(f"  {medal} {i}. {runtime}: {score:.2f}")
    
    print(f"\n🏆 Most Efficient Runtime: {sorted_scores[0][0]}")
    
else:
    print("❌ Cannot perform advanced analysis - no performance data available")

## Summary and Conclusions

This notebook provides a comprehensive comparison of Next.js performance across three JavaScript runtimes. Based on the analysis, you can make informed decisions about which runtime best fits your specific needs.

### Key Insights:

1. **CPU Performance**: Shows which runtime uses the least CPU resources under load
2. **Memory Efficiency**: Identifies the most memory-efficient runtime 
3. **Stability**: Measures consistency of resource usage over time
4. **Health/Reliability**: Tracks application uptime and responsiveness

### How to Use This Analysis:

- **For Production Deployment**: Choose the runtime with the best overall efficiency and stability
- **For Development**: Consider the balance between performance and developer experience
- **For Scaling**: Focus on the runtime with the most predictable resource usage patterns

### Next Steps:

1. **Load Testing**: Run this analysis under different load conditions
2. **Extended Monitoring**: Monitor for longer periods to identify trends
3. **Real-World Scenarios**: Test with actual application workloads
4. **Cost Analysis**: Consider infrastructure costs based on resource usage

The performance characteristics may vary based on your specific application, traffic patterns, and infrastructure setup.